<a href="https://colab.research.google.com/github/anonyme56/ProjetProgrammationDiabete/blob/main/TP_Prog_indian_diabete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prédiction du Diabète - Pipeline de Data Science Complet

## Projet d'Examen - Mini Pipeline de Bout en Bout

**Auteur:** Yohan, Ilias, Chanesse, Aro  

**Date:** Janvier 2026  
**Dataset:** [Pima Indians Diabetes Database](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database)  
**Type de problème:** Classification

---

### Objectif
Prédire si une patiente a le diabète (1) ou non (0) sur la base de mesures diagnostiques médicales.

### Plan du Notebook
1. Acquisition des données
2. Compréhension des données
3. Nettoyage & Prétraitement
4. Analyse Exploratoire des Données (EDA)
5. Modélisation (3+ modèles)
6. Explication des Modèles
7. Optimisation des Hyperparamètres
8. Évaluation & Sélection du Modèle
9. Conclusion & Reproductibilité

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("uciml/pima-indians-diabetes-database")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'pima-indians-diabetes-database' dataset.
Path to dataset files: /kaggle/input/pima-indians-diabetes-database


In [ ]:
# Import des bibliothèques
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

# Configuration
warnings.filterwarnings('ignore')
np.random.seed(42)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Constante pour la reproductibilité
RANDOM_STATE = 42

print("✓ Bibliothèques importées avec succès")
print(f"- NumPy version: {np.__version__}")
print(f"- Pandas version: {pd.__version__}")
print(f"- Matplotlib version: {plt.matplotlib.__version__}")
print(f"- Seaborn version: {sns.__version__}")

✓ Bibliothèques importées avec succès
- NumPy version: 2.0.2
- Pandas version: 2.2.2
- Matplotlib version: 3.10.0
- Seaborn version: 0.13.2
